## Что такое Numba?

Numba – JIT-компилятор (just in time) для Python, который:
- умеет генерировать оптимизированный машинный код с помощью LLVM;
- умеет работать с почти со всеми [python-объектами](https://numba.pydata.org/numba-doc/dev/reference/pysupported.html) и некоторым функционалом [numpy](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html).

Подробнее, что такое LLVM можно посмотреть [тут](https://www.youtube.com/watch?v=PauCAyVg348) или прочитать [тут](https://habr.com/ru/company/huawei/blog/511854/).

**ВАЖНО:** В Numba много циклов – это хорошо!

## Мотивирующий пример

Давайте попробуем найти сумму элементов в двумерном массиве с помощью python.

In [ ]:
import numpy as np

In [ ]:
def sum_array(a):
    n_rows, n_cols = a.shape

    s = 0
    for i in range(n_rows):
        for j in range(n_cols):
            s += a[i, j]
    return s

In [ ]:
a = np.random.random((300, 300))

In [ ]:
%%timeit

_ = sum_array(a)

In [ ]:
%%timeit

_ = a.sum()

А теперь давайте к `numba` :)

In [ ]:
# !pip install numba

In [ ]:
import numba

В `numba` есть специальный декоратор, который позволяет "ускорять" функции – `numba.jit`.

JIT–компилиция, это означает, что для `numba` нужно скопилировать код во время **первого** запуска (он будет медленным), все остальные запуски будут "оптимизированы".

In [ ]:
@numba.jit
def sum_array(a):
    n_rows, n_cols = a.shape

    s = 0
    for i in range(n_rows):
        for j in range(n_cols):
            s += a[i, j]
    return s

In [ ]:
%%time

_ = sum_array(a)

In [ ]:
%%timeit

_ = sum_array(a)

### Заглянем немного внутрь

In [ ]:
@numba.jit
def add(a, b):
    return a + b

In [ ]:
add.inspect_types(), add.inspect_llvm(), add.inspect_asm()

При первом вызове `add` происходит компиляция функции с аргументами данного типа. Numba транслирует python байткод в "промежуточное представление".

In [ ]:
add(1, 1)

In [ ]:
add.inspect_types()

При вызове функции с той же сигнатурой используется уже скомпилированный вариант функции. Если тип входных данных меняется – создается функция с новой сигнатурой.

In [ ]:
add(2, 1)

In [ ]:
add.inspect_types()

In [ ]:
add(1., 1)

In [ ]:
add.inspect_types()

### О режимах запуска в `numba`

Рассмотрим функцию с намеренной ошибкой.

In [ ]:
def fill_array(a):
    result = []
    
    for e in a:
        if e % 2 == 0:
            e = 0
        else:
            e = '1'  # опечтка, здесь дожен быть int
        result.append(e)
        
    return result

In [ ]:
%%time

_ = fill_array(list(range(10_000)))

In [ ]:
@numba.jit
def fill_array(a):
    result = []
    
    for e in a:
        if e % 2 == 0:
            e = 0
        else:
            e = '1'
        result.append(e)
        
    return result

In [ ]:
%%time

_ = fill_array(list(range(10_000)))

При компиляции `fill_array` получаем предупреждение о том, что `numba` запускается в `object` режиме. Смысл проедупреждения в том, что `numba` не может самостоятельно вывести единый тип для данных. Поэтому рекомендуется использовать декоратор `numba.jit(nopython=True)` или `numba.njit`, который бросает исключение вместо предупреждения.

In [ ]:
@numba.njit  # numba.jit(nopython=True)
def fill_array(a):
    result = []
    
    for e in a:
        if e % 2 == 0:
            e = 0
        else:
            e = '1'
        result.append(e)
        
    return result

In [ ]:
_ = fill_array(list(range(10_000)))

Исправим теперь ошибку и сравним время работы функций снова.

In [ ]:
def fill_array(a):
    result = []
    
    for e in a:
        e = int(e % 2 == 0)
        result.append(e)
        
    return result

In [ ]:
%%timeit

_ = fill_array(list(range(10_000)))

In [ ]:
@numba.njit
def fill_array(a):
    result = []
    
    for e in a:
        e = int(e % 2 == 0)
        result.append(e)
        
    return result

In [ ]:
%%timeit

_ = fill_array(list(range(10_000)))

Получаем другое предупреждение, которое говорит нам о том, что `numba` не знает типа данных, которые хранятся в списке `a`. Поэтому при каждом запуске функции придется выводить тип данных. Для того, чтобы корректно работать со списками в `numba` есть типизированные списки. Либо можно использовать numpy массивы.

In [ ]:
a = list(range(10_000))
a_cast = numba.typed.List(a)

a_cast._list_type

In [ ]:
%%timeit

_ = fill_array(a_cast)

In [ ]:
a_np = np.arange(10_000)

In [ ]:
%%timeit

_ = fill_array(a_np)

Сделаем естественную оптимизацию – выделим память под результирущий список и оценим скорость работы.

In [ ]:
@numba.njit
def fill_array(a):
    result = np.empty(len(a), dtype=np.int32)
    
    for i, e in enumerate(a):
        e = int(e % 2 == 0)
        result[i] = e
        
    return result

In [ ]:
%%timeit

_ = fill_array(a_np)

Более того, для `numpy` можно делать векторизированные функции.

In [ ]:
@numba.vectorize
def calc_val(e):
    return int(e % 2 == 0)

calc_val_np = np.vectorize(lambda e: int(e % 2 == 0))

In [ ]:
calc_val(10)

In [ ]:
%%timeit

_ = calc_val(a_np)

In [ ]:
%%timeit

_ = a_np % 2 == 0

In [ ]:
%%timeit

_ = calc_val_np(a_np)

### Не используйте глобальные переменные

In [ ]:
N = 42

In [ ]:
@numba.njit
def magic(a):
    return a + N

magic(np.arange(10))

In [ ]:
N = 0

In [ ]:
magic(np.arange(10))

In [ ]:
magic.recompile()

In [ ]:
magic(np.arange(10))

### Поддержка классов в Numba

Она [есть](https://numba.pydata.org/numba-doc/dev/user/jitclass.html), но в экспериментальном режиме, так что нужно быть аккуратнее. Лучше использовать `numpy` структуры.

Для примера рассмотрим задачу взаимодействия N тел. Для каждого из тел хотим посчитать потенциальную энергию взаимодействия тел:

$$ E_p = -G \frac{m_i m_j}{\left\| r_i - r_j \right\|_2} $$

In [ ]:
ParticleType = np.dtype([
    ('x',  np.float32),
    ('y',  np.float32),
    ('z',  np.float32),
    ('m',  np.float32),
    ('Ep', np.float32),
])

ParticleType

In [ ]:
particles = np.empty(1_000, ParticleType)

particles['x'] = (np.random.random(len(particles)) - 0.5) * 10
particles['y'] = (np.random.random(len(particles)) - 0.5) * 10
particles['z'] = (np.random.random(len(particles)) - 0.5) * 10
particles['m'] = 1.0
particles['Ep'] = 0

In [ ]:
@numba.njit
def distance(a, b):
    return np.sqrt((b.x - a.x) ** 2 + (b.y - a.y) ** 2 + (b.z - a.z) ** 2)


@numba.njit
def solve(particles):
    num = len(particles)
    
    for i in range(0, num):
        for j in range(0, num):
            if i <= j:
                break
            
            p_i, p_j = particles[i], particles[j]
            Ep_delta = p_i.m * p_j.m * distance(p_i, p_j)

            p_i.Ep += Ep_delta
            p_j.Ep += Ep_delta
            
    return particles

In [ ]:
# compile on small data

_ = solve(np.empty(10, ParticleType))

In [ ]:
%%timeit

_ = solve(particles)

In [ ]:
particles['Ep'][:10]

### Ускоряемся дальше

Продолжение в другом ноутбуке.